In [1]:
import os
import sys

proj_root = os.path.dirname(os.path.abspath("."))
# print(proj_root)
sys.path.append(proj_root)

import time
import datetime
import torch
import random
import numpy as np
import os
from pathlib import Path
import argparse

import torch.optim as optim
import torch.nn.functional as F

import gym
import yaml

from minatar_dqn.my_dqn import AgentDQN
from minatar_dqn.replay_buffer import ReplayBuffer
from minatar_dqn.utils.my_logging import setup_logger
from minatar_dqn.models import Conv_QNET, Conv_QNET_one

from experiments.experiment_utils import (
    seed_everything,
    search_files_containing_string,
    split_path_at_substring,
    collect_training_output_files,
)

from minatar_dqn.my_dqn import Conv_QNET, build_environment
from minatar_dqn.redo import apply_redo_parametrization
from experiments.experiment_utils import (
    collect_training_output_files,
    collect_pruning_output_files,
)

from experiments.training.training import read_config_files, get_config_paths

from flatten_dict import flatten
import pandas as pd
import seaborn as sns
import scipy

sns.set()

import plotly

plotly.io.kaleido.scope.mathjax = None

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Setup config
root_dir = os.path.dirname(os.path.abspath("."))

path_experiments_configs = os.path.join(
    root_dir, "experiments", "training", "training_configs"
)
path_experiments_outputs = os.path.join(root_dir, "experiments", "training", "outputs")

default_config_path, experiment_config_paths = get_config_paths(
    path_experiments_configs
)

experiment_configs = read_config_files(default_config_path, experiment_config_paths)

experiment_configs[0]

{'epochs_to_train': 20,
 'seeds': [0],
 'environments': ['breakout'],
 'agent_params': {'agent': 'AgentDQN',
  'args_': {'train_step_cnt': 200000,
   'validation_enabled': True,
   'validation_step_cnt': 125000,
   'validation_epsilon': 0.001,
   'replay_start_size': 5000,
   'batch_size': 32,
   'training_freq': 4,
   'target_model_update_freq': 100,
   'loss_fcn': 'mse_loss',
   'gamma': 0.99,
   'epsilon': {'start': 1.0, 'end': 0.01, 'decay': 250000}}},
 'estimator': {'model': 'Conv_QNET',
  'args_': {'conv_hidden_out_size': 32, 'lin_hidden_out_size': 128}},
 'optim': {'name': 'Adam', 'args_': {'lr': 6.25e-05, 'eps': 0.00015}},
 'replay_buffer': {'max_size': 100000, 'action_dim': 1, 'n_step': 0},
 'redo': {'attach': True,
  'enabled': False,
  'tau': 0.025,
  'beta': 0.1,
  'selection_option': None},
 'reward_perception': None,
 'experiment_name': 'conv32_lin128'}

In [3]:
config = experiment_configs[0]
config["environment"] = "breakout"
config["seed"] = 0

output_path = os.path.join(
    root_dir,
    "experiments",
    "redo",
)

In [4]:
config

{'epochs_to_train': 20,
 'seeds': [0],
 'environments': ['breakout'],
 'agent_params': {'agent': 'AgentDQN',
  'args_': {'train_step_cnt': 200000,
   'validation_enabled': True,
   'validation_step_cnt': 125000,
   'validation_epsilon': 0.001,
   'replay_start_size': 5000,
   'batch_size': 32,
   'training_freq': 4,
   'target_model_update_freq': 100,
   'loss_fcn': 'mse_loss',
   'gamma': 0.99,
   'epsilon': {'start': 1.0, 'end': 0.01, 'decay': 250000}}},
 'estimator': {'model': 'Conv_QNET',
  'args_': {'conv_hidden_out_size': 32, 'lin_hidden_out_size': 128}},
 'optim': {'name': 'Adam', 'args_': {'lr': 6.25e-05, 'eps': 0.00015}},
 'replay_buffer': {'max_size': 100000, 'action_dim': 1, 'n_step': 0},
 'redo': {'attach': True,
  'enabled': False,
  'tau': 0.025,
  'beta': 0.1,
  'selection_option': None},
 'reward_perception': None,
 'experiment_name': 'conv32_lin128',
 'environment': 'breakout',
 'seed': 0}

In [5]:
config["agent_params"]["args_"]["train_step_cnt"] = 20000
config["agent_params"]["args_"]["validation_step_cnt"] = 12500

In [6]:
env_name = config["environment"]

logger = setup_logger(
    env_name=env_name,
    identifier_string="redo_test_experiment",
)
logger.info(
    f'Starting up experiment: {config["experiment_name"]}, environment: {config["environment"]}, seed: {config["seed"]}'
)

### Setup environments ###
train_env = build_environment(
    game_name=config["environment"], random_seed=config["seed"]
)
validation_env = build_environment(
    game_name=config["environment"], random_seed=config["seed"]
)

experiment_agent = AgentDQN(
    train_env=train_env,
    validation_env=validation_env,
    experiment_output_folder=output_path,
    experiment_name="redo_test_experiment",
    resume_training_path=None,
    save_checkpoints=True,
    logger=logger,
    config=config,
)

2023-10-31 21:10:36,179 - root - INFO - redo_test_experiment - Starting up experiment: conv32_lin128, environment: breakout, seed: 0
2023-10-31 21:10:36,185 - root - INFO - redo_test_experiment - Loaded configuration settings.
2023-10-31 21:10:36,193 - root - INFO - redo_test_experiment - Initialized newtworks and optimizer.
2023-10-31 21:10:36,194 - root - INFO - redo_test_experiment - Applied redo parametrization to policy model.
2023-10-31 21:10:36,195 - root - INFO - redo_test_experiment - Applied redo parametrization to target model.


c:\Users\Chainsword\anaconda3\envs\general\lib\site-packages\gym\envs\registration.py:555: UserWarning:

WARN: The environment MinAtar/Breakout-v0 is out of date. You should consider upgrading to version `v1`.

c:\Users\Chainsword\anaconda3\envs\general\lib\site-packages\gym\utils\passive_env_checker.py:20: UserWarning:

WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: bool. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.

c:\Users\Chainsword\anaconda3\envs\general\lib\site-packages\gym\utils\passive_env_checker.py:25: UserWarning:

WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.



In [7]:
experiment_agent.train(train_epochs=20)

2023-10-31 21:10:36,208 - root - INFO - redo_test_experiment - Starting training session at: 0
2023-10-31 21:10:36,209 - root - INFO - redo_test_experiment - Starting training epoch at t = 0


c:\Users\Chainsword\anaconda3\envs\general\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning:

`np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)



2023-10-31 21:11:00,399 - root - INFO - redo_test_experiment - TRAINING STATS | Frames seen: 20000 | Episode: 1796 | Max reward: 5.0 | Avg reward: 0.5027839643652561 | Avg frames (episode): 11.135857461024498 | Avg max Q: -73.82408577866754 | Epsilon: 0.9406 | Train epoch time: 0:00:24.185811
2023-10-31 21:11:00,400 - root - INFO - redo_test_experiment - Starting validation epoch at t = 20000
2023-10-31 21:11:05,722 - root - INFO - redo_test_experiment - VALIDATION STATS | Max reward: 1.0 | Avg reward: 0.46592844974446335 | Avg frames (episode): 10.659284497444634 | Avg max Q: -88.36758111161396 | Validation epoch time: 0:00:05.318992
2023-10-31 21:11:05,722 - root - INFO - redo_test_experiment - Saving checkpoint at t = 20000 ...
2023-10-31 21:11:05,726 - root - DEBUG - redo_test_experiment - Models saved at t = 20000
2023-10-31 21:11:05,732 - root - DEBUG - redo_test_experiment - Training status saved at t = 20000
2023-10-31 21:11:06,980 - root - INFO - redo_test_experiment - Checkpo

True

In [8]:
# collect a set of states to check Q on before and after redo

eval_states = []
for i in range(100):
    # get a random action from the environment
    action = experiment_agent.train_env.action_space.sample()
    s_prime, reward, is_terminated, truncated, info = experiment_agent.train_env.step(
        action
    )
    s_prime = torch.tensor(s_prime, device="cpu").float()

    if is_terminated:
        experiment_agent.train_env.reset()
        continue

    else:
        eval_states.append(s_prime)

In [19]:
import copy

eval_states_tensor = torch.stack(eval_states)

predictions_init = experiment_agent.policy_model(eval_states_tensor)
predictions_init

max_q_vals_init = []
for state in eval_states_tensor:
    max_q_val = experiment_agent.get_max_q_val_for_state(state.unsqueeze(0))
    max_q_vals_init.append(max_q_val)

# get the weights before redo
state_dict_init = copy.deepcopy(experiment_agent.policy_model.state_dict())

In [12]:
# apply redo
reset_indexes = experiment_agent.policy_model.apply_redo()
reset_indexes

[tensor([ 3,  4,  6, 15, 18, 19, 20, 21, 22, 24, 25, 28]),
 tensor([ 2,  4,  5,  6,  7, 12, 14, 15, 16, 22, 29, 30, 31]),
 tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  13,  14,
          15,  16,  17,  18,  19,  20,  22,  23,  24,  25,  26,  27,  28,  29,
          30,  32,  34,  35,  36,  37,  38,  39,  40,  42,  44,  45,  46,  47,
          48,  50,  51,  53,  54,  55,  56,  58,  59,  60,  61,  62,  63,  64,
          65,  67,  68,  69,  71,  72,  75,  76,  77,  78,  79,  80,  81,  82,
          83,  84,  86,  87,  88,  89,  91,  92,  94,  95,  96,  97,  98,  99,
         100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 113, 114, 115,
         116, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127])]

In [24]:
# get the difference between the values before and after
predictions_after = experiment_agent.policy_model(eval_states_tensor)
predictions_after

max_q_vals_after = []
for state in eval_states_tensor:
    max_q_val = experiment_agent.get_max_q_val_for_state(state.unsqueeze(0))
    max_q_vals_after.append(max_q_val)
    
# get the weights before redo
state_dict_after = copy.deepcopy(experiment_agent.policy_model.state_dict())

In [23]:
diff = np.array(max_q_vals_init) - np.array(max_q_vals_after)
sum(diff)

0.0

In [28]:
for layer in state_dict_after:
    if not torch.equal(state_dict_init[layer], state_dict_after[layer]):
        print(f"Weights changed in layer: {layer}")

Weights changed in layer: features.1.running_avg
Weights changed in layer: features.1.running_avg_cnt
Weights changed in layer: features.3.running_avg
Weights changed in layer: features.3.running_avg_cnt
Weights changed in layer: fc.1.running_avg
Weights changed in layer: fc.1.running_avg_cnt
